# Fields constrain

In [1]:
import warnings
warnings.filterwarnings('ignore')

## The problems
A very common scenario that we face when working with tabular data is finding columns that have very particular relationships between them which are very hard to model and easily confuse the Tabular Models.

Some simple examples of these scenarios include:
- A table that has the columns `country` and `city`: In such scenario, it might be very hard to learn which `country` each `city` belongs to, and when sampling probabilistically, the model is likely to end up generating invalid `country`/`city` combinations.
- A table that contains both the `ag`e and the `date of birth` of a user. The model will learn the `age` and `date of birth` distributions and mostly generate valid combinations, but in some cases it might end up giving back `ages` **that do not correspond to the** `given date of birth`.

These kind of special relationships between columns are called Constraints.


In [2]:
from sdv.demo import load_tabular_demo

employees = load_tabular_demo()
employees

,company,department,employee_id,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
0,Pear,Sales,1,34,27,7,78500.00,10500.00,3,1.0,0.0,0.0
1,Pear,Design,5,41,40,1,45960.55,11115.47,2,0.0,0.0,1.0
2,Glasses,AI,1,45,37,8,58500.00,6000.00,5,1.0,0.0,0.0
3,Glasses,Search Engine,7,38,36,2,121489.44,15194.26,3,0.0,0.0,1.0
4,Cheerper,BigData,6,48,45,3,33000.00,23500.00,4,0.0,1.0,0.0
5,Cheerper,Support,11,42,33,9,36500.00,9000.00,3,0.0,1.0,0.0
6,Pear,Sales,28,49,43,6,48500.00,6500.00,5,1.0,0.0,0.0
7,Pear,Design,75,45,39,6,91072.47,19286.46,5,0.0,0.0,1.0
8,Glasses,AI,33,35,32,3,49000.00,7500.00,3,1.0,0.0,0.0
9,Glasses,Search Engine,56,44,37,7,62711.45,24441.71,2,0.0,0.0,1.0


## How does SDV Handle Constraints?

### Transform Strategy
When using this strategy, SDV applies a transformation to the data before learning it in a way that allows the model to better capture the data properties. For example, if we have one column that needs to be always greater than the other one, SDV can do the following:
- Replace the higher column with the difference between the two columns, which will always be positive.
- Model the transformed data and sample new values.
- Recompute the value of the high column by adding the values of the lower column to it.

The Transform strategy is very efficient and does not affect the speed of the modeling and sampling process, but in some cases might affect the quality of the learning process or simply not be possible.

### Reject Sampling Strategy
In the cases where applying a Transform strategy is not possible or may affect the quality of the learning process, SDV can apply a Reject Sampling strategy.

When using this strategy, SDV validates the sampled rows, discards the ones that do not adjust to the constraint, and re-samples them. This process is repeated until enough rows have been sampled.


## SVD Framework Predefined Constraints

### Unique Constraint

In [3]:
from sdv.constraints import Unique

unique_employee_id_company_constraint = Unique(columns=['employee_id', 'company'])

### UniqueCombinations Constraint

In [4]:
from sdv.constraints import UniqueCombinations

unique_company_department_constraint = UniqueCombinations(columns=['company', 'department'],
                                                          handling_strategy='transform', # 'reject_sampling'
                                                         )

### GreaterThan Constraint

In [5]:
from sdv.constraints import GreaterThan

age_gt_age_when_joined_constraint = GreaterThan(low='age_when_joined',
                                                high='age',
                                                handling_strategy='reject_sampling'
                                               )

**Lower or higher than a scalar value or specific datetime**

In [6]:
salary_gt_30000_constraint = GreaterThan(low=30000,
                                         high=['salary'], # or 'salary'
                                         scalar='low',
                                         handling_strategy='reject_sampling'
                                        )

### Positive and Negative Constraints

In [7]:
from sdv.constraints import Positive

positive_age_constraint = Positive(columns='age',
                                   strict=False,
                                   handling_strategy='reject_sampling'
                                  )

### ColumnFormula Constraint

In [8]:
from sdv.constraints import ColumnFormula

In [9]:
def years_in_the_company(data):
    return data['age'] - data['age_when_joined']

In [10]:
years_in_the_company_constraint = ColumnFormula(column='years_in_the_company',
                                                formula=years_in_the_company,
                                                handling_strategy='transform'
                                               )

### Rounding Constraint

In [11]:
from sdv.constraints import Rounding

salary_rounding_constraint = Rounding(columns='salary',
                                      digits=2,
                                      handling_strategy='transform'
                                     )

### Between Constraint

In [12]:
from sdv.constraints import Between

reasonable_age_constraint = Between(column='age',
                                    low=15,
                                    high=90,
                                    handling_strategy='transform'
                                   )

### OneHotEncoding Constraint

In [13]:
from sdv.constraints import OneHotEncoding

one_hot_constraint = OneHotEncoding(columns=['full_time', 'part_time', 'contractor'])

## Using the CustomConstraint

In [14]:
from sdv.constraints import CustomConstraint

### Apply constraint to one column

In [15]:
def transform(table_data):
    base = 500.
    table_data['salary'] = table_data['salary'] / base
    return table_data

In [16]:
def reverse_transform(table_data):
    base = 500.
    table_data['salary'] = table_data['salary'].round() * base
    return table_data

In [17]:
custome_column_constraint = CustomConstraint(transform=transform, reverse_transform=reverse_transform)

### Constraint to multiple columns

In [18]:
def transform(column_data):
    base = 500.
    return column_data / base

In [19]:
def reverse_transform(column_data):
    base = 500.
    return column_data.round() * base

In [20]:
multiple_columns_constraint = CustomConstraint(columns=['salary', 'annual_bonus'],
                              transform=transform,
                              reverse_transform=reverse_transform
                             )

### Constrain with condition from other columns

In [21]:
def transform(table_data, column):
    base = 500.
    table_data[column] = table_data[column] / base
    return table_data

In [22]:
def reverse_transform(table_data, column):
    base = 500.
    is_not_contractor = table_data["contractor"] == 0.
    table_data[column] = table_data[column].round(4)
    table_data[column].loc[is_not_contractor] = table_data[column].loc[is_not_contractor].round()
    table_data[column] *= base
    return table_data

In [23]:
multiple_columns_with_condition_constraint = CustomConstraint(columns=['salary', 'annual_bonus'],
                                                              transform=transform,
                                                              reverse_transform=reverse_transform
                                                             )

### Constrain based on reject sampling

In [24]:
def is_valid(table_data):
    base = 500.
    return table_data['salary'] % base == 0

def is_valid(column_data):
    base = 500.
    return column_data % base == 0

def is_valid(table_data, column):
    base = 500.
    is_contractor = table_data.contractor == 1
    valid = table_data[column] % base == 0
    contractor_salary = employees['salary'].loc[is_contractor]
    valid.loc[is_contractor] = contractor_salary == contractor_salary.round(2)
    return valid

In [25]:
custom_reject_constraint = CustomConstraint(
                                            columns=['salary', 'annual_bonus'],
                                            is_valid=is_valid
                                           )

## Using the Constraints in Synthesize model

In [26]:
from sdv.tabular import GaussianCopula

constraints = [unique_employee_id_company_constraint,
               unique_company_department_constraint,
               age_gt_age_when_joined_constraint,
               salary_gt_30000_constraint,
               positive_age_constraint,
               years_in_the_company_constraint,
               salary_rounding_constraint,
               reasonable_age_constraint,
               one_hot_constraint,
               multiple_columns_with_condition_constraint,
#                custom_reject_constraint,
              ]

model = GaussianCopula(constraints=constraints)

In [27]:
model.fit(employees)

In [28]:
sampled = model.sample(100)

In [29]:
sampled

,company,department,employee_id,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
0,Cheerper,BigData,27,42,41,1,63000.00,24500.00,4,0.0,1.0,0.0
1,Cheerper,BigData,42,43,39,4,117500.00,10500.00,3,0.0,1.0,0.0
2,Pear,Design,6,41,38,3,84000.00,15000.00,4,1.0,0.0,0.0
3,Cheerper,BigData,62,59,45,14,55439.95,15174.65,4,0.0,1.0,0.0
4,Cheerper,BigData,30,39,38,1,42870.35,24441.70,2,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Glasses,Search Engine,70,37,33,4,89876.45,13275.85,3,0.0,0.0,1.0
97,Pear,Sales,19,42,36,6,68145.65,8015.80,4,1.0,0.0,0.0
99,Cheerper,BigData,24,37,35,2,97000.00,9500.00,3,0.0,1.0,0.0
101,Glasses,AI,25,42,39,3,33000.00,17104.45,3,0.0,0.0,1.0
